In [11]:
rm(list = ls())
setwd('./')

library(GSVA)
library(GEOquery)

In [12]:
prepeocessing <- function(file, output, gpl_num="GPL570"){
    print(file)
    expr.df <- read.table(file, header = TRUE, sep = "\t")
    gpl <- getGEO(gpl_num,destdir ="./data/raw")
    
    probe <- Table(gpl)[,c(1,11)]
    colnames(probe) <- c("ID", "GeneSymbol")
    probe <- probe[!is.na(probe$GeneSymbol),]
    probe <- probe[-c(which(probe$GeneSymbol==""),
                    grep("///", probe$GeneSymbol), which(probe$GeneSymbol==NA)),]
    probe <- probe[!duplicated(probe$ID),]
    row.names(probe) <- probe[, 1]
    same <- intersect(probe[, 1], row.names(expr.df))
    expr.df <- expr.df[same,]
    probe <- probe[same,]
    symbol <- sapply(1:dim(expr.df)[1], 
                function(i) {
                    as.character(probe[which(probe[, 1]==rownames(expr.df)[i]), 2])})

    expr.df_1 <- data.frame(symbol, expr.df, stringsAsFactors = FALSE)
    print('aggregating')
    expr.df_2 <- aggregate(x = expr.df_1[, 2:ncol(expr.df_1)], by = list(expr.df_1[, 1]), FUN = mean)
    if (!is.null(expr.df_2)) { colnames(expr.df_2)[1] <- "Gene" }
    rownames(expr.df_2) <- expr.df_2$Gene
    expr.df_2 <- expr.df_2[,-1]
    ex <- expr.df_2
    qx <- as.numeric(quantile(ex, c(0., 0.25, 0.5, 0.75, 0.99, 1.0), na.rm=T))
    LogC <- (qx[5] > 100) ||
    (qx[6]-qx[1] > 50 && qx[2] > 0) ||
    (qx[2] > 0 && qx[2] < 1 && qx[4] > 1 && qx[4] < 2)

    if (LogC) { ex[which(ex <= 0)] <- NaN
    ex <- log2(ex)
    print("log2 transform finished")}else{print("log2 transform not needed")}
    print(output)
    save(ex,file=output)
}


In [ ]:
file = './data/raw/CMS_DATA/GSE14333_frma_expression.tsv'
output = './data/processed/GSE14333_preprocessed.Rdata'
prepeocessing(file, output)

file = './data/raw/CMS_DATA/GSE39582_frma_expression.tsv'
output = './data/processed/GSE39582_preprocessed.Rdata'
prepeocessing(file, output)

file = './data/raw/CMS_DATA/GSE33113_frma_expression.tsv'
output = './data/processed/GSE33113_preprocessed.Rdata'
prepeocessing(file, output)

file = './data/raw/CMS_DATA/GSE37892_frma_expression.tsv'
output = './data/processed/GSE37892_preprocessed.Rdata'
prepeocessing(file, output)

In [13]:
load('./data/processed/str_scissor_degs.Rdata')
degs$gene <- rownames(degs)
degs$v <- -log10(degs$p_val_adj)
degs$pct <- (degs$pct.1 - degs$pct.2)
degs$group <- ifelse((degs$p_val_adj < 0.05) & (degs$avg_log2FC > 1.5),'Up','Down/NS')
sig_degs <- subset(degs, p_val_adj < 0.05 & degs$avg_log2FC > 1.5)

gs <- list(scSTR=rownames(sig_degs))

In [20]:
file <- './data/raw/CMS_DATA/clinical_molecular_public_all_add_14333.txt'
clinical_df <- read.table(file, header = TRUE, sep = "\t")

#GSE14333
load('./data/processed/GSE14333_preprocessed.Rdata')
dim(ex)
stopifnot(length(ex) == 290)
ex <- as.matrix(ex)

stopifnot(length(gs[[1]]) == 22)

scSTR <- gsva(ex, gs)
scSTR <- t(scSTR)

rownames(scSTR) <- gsub("\\.CEL", "", rownames(scSTR))

merged_data <- clinical_df
merged_data <- merge(scSTR, merged_data, by.x = "row.names", by.y = "sample", all.x = TRUE)

output = './data/processed/GSVA_GSE14333.csv'
write.table(merged_data, file = output, sep = ",")


[1] 21655   290

Estimating GSVA scores for 1 gene sets.
Estimating ECDFs with Gaussian kernels
  |======================================================================| 100%



In [23]:
df <- data.frame(
  sample = c("GSM820048", "GSM820049", "GSM820050", "GSM820051", "GSM820052", "GSM820053", "GSM820054", "GSM820055", "GSM820056", "GSM820057",
             "GSM820058", "GSM820059", "GSM820060", "GSM820061", "GSM820062", "GSM820063", "GSM820064", "GSM820065", "GSM820066", "GSM820067",
             "GSM820068", "GSM820069", "GSM820070", "GSM820071", "GSM820072", "GSM820073", "GSM820074", "GSM820075", "GSM820076", "GSM820077",
             "GSM820078", "GSM820079", "GSM820080", "GSM820081", "GSM820082", "GSM820083", "GSM820084", "GSM820085", "GSM820086", "GSM820087",
             "GSM820088", "GSM820089", "GSM820090", "GSM820091", "GSM820092", "GSM820093", "GSM820094", "GSM820095", "GSM820096", "GSM820097",
             "GSM820098", "GSM820099", "GSM820100", "GSM820101", "GSM820102", "GSM820103", "GSM820104", "GSM820105", "GSM820106", "GSM820107",
             "GSM820108", "GSM820109", "GSM820110", "GSM820111", "GSM820112", "GSM820113", "GSM820114", "GSM820115", "GSM820116", "GSM820117",
             "GSM820118", "GSM820119", "GSM820120", "GSM820121", "GSM820122", "GSM820123", "GSM820124", "GSM820125", "GSM820126", "GSM820127",
             "GSM820128", "GSM820129", "GSM820130", "GSM820131", "GSM820132", "GSM820133", "GSM820134", "GSM820135", "GSM820136", "GSM820137"),
  Column = c("col001", "col002", "col003", "col004", "col005", "col006", "col007", "col008", "col009", "col010",
             "col011", "col012", "col013", "col014", "col015", "col016", "col017", "col018", "col019", "col020",
             "col021", "col022", "col023", "col024", "col026", "col033", "col034", "col035", "col036", "col037",
             "col038", "col039", "col040", "col041", "col044", "col045", "col046", "col047", "col048", "col049",
             "col050", "col051", "col052", "col053", "col054", "col055", "col056", "col057", "col058", "col059",
             "col060", "col061", "col062", "col063", "col064", "col065", "col066", "col067", "col068", "col069",
             "col070", "col072", "col073", "col074", "col075", "col076", "col077", "col078", "col079", "col080",
             "col081", "col082", "col083", "col084", "col085", "col086", "col089", "col090", "col091", "col092",
             "col093", "col094", "col095", "col096", "col097", "col098", "col099", "col100", "col101", "col102")
)

load('./data/processed/GSE33113_preprocessed.Rdata')
dim(ex)
stopifnot(length(ex) == 96)
result_df <- dplyr::left_join(df, clinical_df, by = c("Column" = "sample"))
dim(result_df)
result_df <- result_df[, !names(result_df) %in% "Column"]
ex <- as.matrix(ex)

gs <- list(scSTR=rownames(sig_degs))
stopifnot(length(gs[[1]]) == 22)

scSTR <- gsva(ex, gs)
scSTR <- t(scSTR)

rownames(scSTR) <- gsub("\\.CEL", "", rownames(scSTR))

merged_data <- result_df
merged_data <- merge(scSTR, merged_data, by.x = "row.names", by.y = "sample", all.x = TRUE)

output = './data/processed/GSVA_GSE33113.csv'
write.table(merged_data, file = output, sep = ",")

[1] 21655    96

[1] 90 20

Estimating GSVA scores for 1 gene sets.
Estimating ECDFs with Gaussian kernels
  |======================================================================| 100%



In [24]:
load('./data/processed/GSE37892_preprocessed.Rdata')
dim(ex)
stopifnot(length(ex) == 130)
ex <- as.matrix(ex)

stopifnot(length(gs[[1]]) == 22)
scSTR <- gsva(ex, gs)
scSTR <- t(scSTR)
rownames(scSTR) <- gsub("_.*", "", rownames(scSTR))

merged_data <- clinical_df
merged_data <- merge(scSTR, merged_data, by.x = "row.names", by.y = "sample", all.x = TRUE)

output = './data/processed/GSVA_GSE37892.csv'
write.table(merged_data, file = output, sep = ",")

[1] 21655   130

Estimating GSVA scores for 1 gene sets.
Estimating ECDFs with Gaussian kernels
  |======================================================================| 100%



In [25]:
load('./data/processed/GSE39582_preprocessed.Rdata')
dim(ex)
stopifnot(length(ex) == 566)
ex <- as.matrix(ex)

stopifnot(length(gs[[1]]) == 22)
scSTR <- gsva(ex, gs)
scSTR <- t(scSTR)
rownames(scSTR) <- gsub("_.*", "", rownames(scSTR))

merged_data <- clinical_df
merged_data <- merge(scSTR, merged_data, by.x = "row.names", by.y = "sample", all.x = TRUE)

output = './data/processed/GSVA_GSE39582.csv'
write.table(merged_data, file = output, sep = ",")

[1] 21655   566

Estimating GSVA scores for 1 gene sets.
Estimating ECDFs with Gaussian kernels
  |======================================================================| 100%



In [29]:
load('./data/raw/crc_bulk_tpm.Rdata')
load('./data/raw/crc_bulk_survival.Rdata')

ex <- as.matrix(log2(bulk_dataset_1+1))
dim(ex)

stopifnot(dim(ex)[2] == 576)

gs <- list(scSTR=rownames(sig_degs))
stopifnot(length(gs[[1]]) == 22)
scSTR <- gsva(ex, gs)
scSTR <- t(scSTR)
rownames(scSTR) <- gsub("_.*", "", rownames(scSTR))
merged_data <- merge(scSTR, bulk_survival, by.x = "row.names", by.y = "TCGA_patient_barcode", all.x = TRUE)
dim(merged_data)
write.table(merged_data, file = "./data/processed/GSVA_TCGA_CRC_Gaussian.csv",sep = ",")

[1] 59427   576

Warning message in .filterFeatures(expr, method):
“3514 genes with constant expression values throuhgout the samples.”
Warning message in .filterFeatures(expr, method):
“Since argument method!="ssgsea", genes with constant expression values are discarded.”


Estimating GSVA scores for 1 gene sets.
Estimating ECDFs with Gaussian kernels
  |======================================================================| 100%



[1] 576   4

In [30]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Debian GNU/Linux 12 (bookworm)

Matrix products: default
BLAS/LAPACK: ./anaconda3/envs/scCRC/lib/libopenblasp-r0.3.24.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Berlin
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] GSVA_1.48.3         GEOquery_2.68.0     Biobase_2.60.0     
[4] BiocGenerics_0.46.0

loaded via a namespace (and not attached):
 [1] tidyselect_1.2.0            IRdisplay_1.1              
 [3] dplyr_1.1.3                